In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
import torch
from torch import nn



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# NOTE, THIS SESSION IS SET UP TO RUN BIDERCTIONAL LSTM:

# IN ORDER TO RUN RNN: 
    #1. UNCOMMENT self.encoder = nn.RNN() AND COMMENT SIBLING VARIABLE
    #2. UNCOMMENT output, hidden = self.encoder(embeddings) AND COMMENT SIBLING VARIABLE
    #3. UNCOMMENT net = SimpleRNN(vocab, 100, 256, 1, 1) AND COMMENT SIBLING VARIABLE

# IN ORDER TO RUN LSTM: 
    #1. UNCOMMENT net = SimpleRNN(vocab, 100, 256, 1, 1) AND COMMENT SIBLING VARIABLE
    

##### device = "cuda" if torch.cuda.is_available() else "cpu"
#print("Using {} device".format(device))


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
d = pd.read_csv('../input/selective-stock-headlines-sentiment/Project6500.csv')
glove = pd.read_csv('../input/global-vector/glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
#print(d['datetime'].dtype)

In [ ]:
d['datetime'] = d['datetime'].astype(str).str[:10]

# convert to date type the column transforming the errors into Null values
d['datetime'] = pd.to_datetime(d['datetime'], errors='coerce')
#Drop rows with Null value
d.dropna()
d = d.sort_values(by='datetime')
td = d.copy()

#train sample
rg = (d['datetime'] <= '2019-08-30')
d = d.loc[rg]

#test sample
trg = (td['datetime'] > '2019-08-30')
td = td.loc[trg]






In [ ]:
#Assign values to label and text arrays

label = []
text = []
for i in range(len(d)):
    text.append(d.iloc[i][1])
    label.append(d.iloc[i][3])

    
test_text = [] 
test_label = []
for i in range(len(td)):
    test_text.append(td.iloc[i][1])
    test_label.append(td.iloc[i][3])
    


In [ ]:
# a bit of text cleaning
listOfWords = []
def transformer(text):
    for num, sent in enumerate(text):
        sent = re.sub('[^a-zA-z]+', ' ', sent).lower()
        tokens = nltk.word_tokenize(sent)
        for t in tokens:
            listOfWords.append(t)
        text.insert(num, tokens)
        text.pop(num+1)

# get the list of words for both test and train data
transformer(test_text)
testListOfWords = listOfWords
listOfWords = []
transformer(text)

#num of unique words
uniqueWrds = set(listOfWords)
vocab = len(set(listOfWords))
print(vocab)

len(listOfWords)//(10*30)
#num of unique labels
classes = len(set(label))


index_to_wrd = dict(enumerate(uniqueWrds))


word_to_index = {i[1] : i[0] for i in index_to_wrd.items()}



# Total number of words
encoded = np.array([word_to_index[wrd] for wrd in listOfWords])



In [ ]:
def findMaxLen():
    list_len = [len(l) for l in text]
    return max(list_len)
#print(findMaxLen())

In [ ]:
# n_labels is number of unique words
def one_hot_encode(arr, n_labels):

   
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.int64)
    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1
    
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:

#num_batches = len(text)//(sentence_length*num_seqs)
    

def batches(text, label, num_seqs=32):

    counter = 0
    #create empty arrays with the specified number of columns 
    x = np.array([], dtype=int).reshape(0, findMaxLen())
    y = np.array([], dtype=int).reshape(0,1)
    
    for sent, l in zip(text, label):
        # create a np array with zeros with length 30
        tmp1 = np.zeros((findMaxLen()), dtype=int)
        #tmp1 = np.random.randint(0, high=vocab,size=findMaxLen(), dtype=int)
        #create a 1d array
        tmp2 = np.atleast_1d(np.array(l))
        
        for ind, wrd in enumerate(sent):
            if wrd in uniqueWrds and ind < 30:
                tmp1[ind] = word_to_index[wrd]
        # the arrays to the empty arrays
        x = np.vstack([x, tmp1])
        y = np.vstack([y, tmp2])
        counter +=1
        if counter == num_seqs:
            yield x, np.squeeze(y,1)
            counter = 0
            x = np.array([], dtype=int).reshape(0, findMaxLen())
            y = np.array([], dtype=int).reshape(0,1)
      

In [ ]:
#Dictionary of word to embedding array
embedding_dict = {key : val.values for key, val in glove.T.items()}

#Create a matrix for one embedding for each word in the training dataset
def embeddingMatrix(word_index, embedding_dict, dimensions):
    e_matrix = torch.zeros((len(word_index), dimensions))
    
    for wrd, ind in word_index.items():
        if wrd in embedding_dict:
            e_matrix[ind] = torch.from_numpy(embedding_dict[wrd])
    
    return e_matrix


In [ ]:
embedding_matrix = embeddingMatrix(word_to_index, embedding_dict, 100)
print(embedding_matrix.shape)

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers, dropout):
        #calling the init function of the RNN parent
        super(RNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix)
        
        #self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        
        self.encoder = nn.LSTM(embed_dim,
                               hidden_dim,
                               n_layers,
                               batch_first=True,
                               dropout=dropout,
                               bidirectional=True
                              )
        
        #Linear transformation
        self.decoder = nn.Linear(hidden_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    
    def forward(self, inputs):
        
         #output-(batch_size, timesteps, embed_dim)
        embeddings = self.dropout(self.embedding(inputs))
        
       
        #embeddings = torch.max(embeddings, dim=2)
       
        embeddings = embeddings.type(torch.cuda.FloatTensor)
        #embeddings = embeddings[0].type(torch.cuda.FloatTensor)
        
        #output of each timestep
        #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden, cell) = self.encoder(embeddings)
        
        merge = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        
       
        output = self.decoder(merge)
        
        return output
        
    
    

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers):
        #calling the init function of the RNN parent
        super(SimpleRNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        self.encoder = nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first=True)
        
        #self.encoder = nn.RNN(embed_dim, hidden_dim, n_layers, batch_first=True)
        
        #Linear transformation
        self.decoder = nn.Linear(hidden_dim, output_dim)
        
       
    
    def forward(self, inputs):
        
        
        #embeddings[batch_size, timesteps, vocab_size, embed_dim]
        embeddings = self.embedding(inputs)
        
        #embeddings[batch_size, timesteps, embed_dim]
        #embeddings = torch.max(embeddings, dim=2)
        
        embeddings = embeddings.type(torch.cuda.FloatTensor)
        #embeddings = embeddings[0].type(torch.cuda.FloatTensor)
        
        #output represents the concatenation of all hidden states
        #hidden represents the final hidden state
        output, (hidden, cell) = self.encoder(embeddings)
        #output, hidden = self.encoder(embeddings)
        

        hidden = hidden.squeeze(0)
        
        
        return self.decoder(hidden)
    

In [ ]:
net = RNN(vocab, 100, 256, 1, 2, 0.2)
#net = SimpleRNN(vocab, 100, 256, 1, 1)
net.to(device)

In [ ]:
def binaryAccuracy(pred, target):
    x = torch.round(torch.sigmoid(pred))
    #for each target and x value if both are equal than +1 else 0
    correct = (x == target).float()
    acc = correct.sum() / len(correct)
    return acc
    

In [ ]:
def train(model, text, label, epochs, lr=0.001):
    
    model.train()
    
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    #The BCEWithLogitsLoss criterion carries out both the sigmoid and the binary cross entropy steps
    criterion = nn.BCEWithLogitsLoss()
    

    for e in range(epochs+1):
        if e > 0:
            
            print("Epoch {}/{}".format(e, epochs), 
                "Loss: {}".format(accumLoss/counter),
                "accuracy: {}".format(accumAcc/counter))
        counter = 0
        accumLoss = 0      
        accumAcc = 0
        if e > 10:
            break
        #h = model.init_hidden(10)
        for x, y in batches(text, label):
            
            #Avoid backpropagating through the entire history
            #h = tuple([each.data for each in h])
            
            #x = one_hot_encode(x, vocab)
            
            x = torch.from_numpy(x).to(device)
            
            #shape(10,1)
            output = model(x)
            
           
            y = torch.from_numpy(y)
           
            y = torch.unsqueeze(y,1).to(device)
           
            
            loss = criterion(output, y.float())
            accumLoss += loss
           
            acc = binaryAccuracy(output,y)
            accumAcc += acc
            
            #Avoids gradient accumulation this way gradients don't point in the wrong direction
            opt.zero_grad()
            #computes the gradients of all the parameters in loss that require gradient 
            loss.backward()
            #Updates all the parameters based on parameters grad
            opt.step()
            
            
            counter += 1
        
            #print("Epoch {}/{}".format(e+1, epochs), 
            #"Loss: {}".format(loss.item()),
            #"accuracy: {}".format(acc))
    

In [ ]:
def predict(model, text, labels):
    
    counter = 0
    acc= 0
    for x, y in batches(text, test_label):
        #for i in x:
           
            #turn the 1d array into 2d with 1 at the beginning of th array
            #i = np.expand_dims(i, axis=0)
       
        #x = one_hot_encode(x, vocab)
        
        inputs = torch.from_numpy(x).to(device)
        
        y = torch.from_numpy(y)
           
        y = torch.unsqueeze(y,1).to(device)
        #shape(10,1)
        outputs = model(inputs)
        acc += binaryAccuracy(outputs, y)
        counter += 1
        
    print(acc/counter)    
        #prediction = torch.sigmoid(outputs)
        #for p in prediction:
            #if p.round() == 1:
                #print('positive')
            #else:
                #print('negative')
        
        


In [ ]:
train(net, text, label, 10)

In [ ]:
predict(net, test_text, test_label)